In [8]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
# nltk.download("all")
import matplotlib.pyplot as plt
import torch

%matplotlib inline

# Data loading
Fetch spam detection dataset from Huggingface and load parquet files.

In [1]:
!pip install fastparquet

   ---------------------------------------- 0.0/673.3 kB ? eta -:--:--
   --------------- ------------------------ 262.1/673.3 kB ? eta -:--:--
   ------------------------------- -------- 524.3/673.3 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 673.3/673.3 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 599.9 kB/s eta 0:00:02
   ------------------ --------------------- 0.8/1.7 MB 799.2 kB/s eta 0:00:02
   ------------------------ --------------- 1.0/1.7 MB 1.0 MB/s eta 0:00:01
   ------------------------------------- -- 1.6/1.7 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset('Deysi/spam-detection-dataset')

# Downloaded parquet files can be accessed via dataset['train'] and dataset['test']
train_path = dataset['train'].cache_files[0]['filename']
test_path = dataset['test'].cache_files[0]['filename']


parquet_data = dataset['train'].to_pandas()
parquet_data.head()

,text,label
0,hey I am looking for Xray baggage datasets can...,not_spam
1,"""Get rich quick! Make millions in just days wi...",spam
2,URGENT MESSAGE: YOU WON'T BELIEVE WHAT WE HAVE...,spam
3,[Google AI Blog: Contributing Data to Deepfake...,not_spam
4,Trying to see if anyone already has timestamps...,not_spam


# Create data sets
Create numpy arrays: texts and labels.

In [5]:
texts = parquet_data['text'].values
labels = parquet_data['label'].apply(lambda x: 1 if x == 'spam' else 0).values
assert(len(texts) == len(labels))

# Download FastText vectors
Download and unzip crawl-300d-2M.vec.zip from https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip and place in the notebook directory.

# Sentence pre-processing
Tokenize, build vocabulary, and encode texts.

In [6]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def tokenize(texts):
    max_len = 0
    tokenized_texts = []
    word2idx = {'<pad>': 0, '<unk>': 1}
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(str(sent).lower())
        tokenized_texts.append(tokenized_sent)
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1
        max_len = max(max_len, len(tokenized_sent))
    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    input_ids = []
    for tokenized_sent in tokenized_texts:
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))
        input_id = [word2idx.get(token, word2idx['<unk>']) for token in tokenized_sent]
        input_ids.append(input_id)
    return np.array(input_ids)

In [9]:
print('Tokenizing...\n')
tokenized_texts, word2idx, max_len = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, max_len)
print(f'Vocab size: {len(word2idx)}, Max length: {max_len}')

Tokenizing...

Vocab size: 25959, Max length: 8322
Vocab size: 25959, Max length: 8322


# Load pretrained FastText vectors

In [10]:
from tqdm.notebook import tqdm

def load_pretrained_vectors(word2idx, fname):
    print('Loading pretrained vectors...')
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))
    count = 0
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)
    print(f'There are {count} / {len(word2idx)} pretrained vectors found.')
    return embeddings

In [11]:
# Place crawl-300d-2M.vec in the working directory
embeddings = load_pretrained_vectors(word2idx, 'crawl-300d-2M.vec')
embeddings = torch.tensor(embeddings)

Loading pretrained vectors...


0it [00:00, ?it/s]

There are 16646 / 25959 pretrained vectors found.


# PyTorch data loaders

In [12]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, SequentialSampler)

def data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=10):
    train_inputs = torch.tensor(train_inputs, dtype=torch.long)
    val_inputs = torch.tensor(val_inputs, dtype=torch.long)
    train_labels = torch.tensor(train_labels, dtype=torch.long)
    val_labels = torch.tensor(val_labels, dtype=torch.long)

    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [13]:
from sklearn.model_selection import train_test_split

# Train Test Split with test set size = 5%
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.05, random_state=42)

# Load data to PyTorch DataLoader, batch_size = 50
train_dataloader, val_dataloader = data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

# CNN Model
Define the CNN_NLP model class.

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        super(CNN_NLP, self).__init__()
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding, freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim, out_channels=num_filters[i], kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(sum(num_filters), num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids):
        x_embed = self.embedding(input_ids).float()
        x_reshaped = x_embed.permute(0, 2, 1)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in x_conv_list]
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list], dim=1)
        logits = self.fc(self.dropout(x_fc))
        return logits

# Set device

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optimizer and Model Initialization

In [16]:
import torch.optim as optim

def initialize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and num_filters need to be of the same length."
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=num_classes,
                        dropout=dropout)
    cnn_model.to(device)
    optimizer = optim.RMSprop(cnn_model.parameters(), lr=learning_rate)
    return cnn_model, optimizer

# Training loop and evaluation

In [22]:
import random
import time
from tqdm import tqdm

loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    best_accuracy = 0
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)
    for epoch_i in range(epochs):
        t0_epoch = time.time()
        total_loss = 0
        model.train()
        for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch_i+1}")):
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)
            optimizer.zero_grad()
            logits = model(b_input_ids)
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            if (step + 1) % 5 == 0:
                preds = torch.argmax(logits, dim=1).flatten()
                acc = (preds == b_labels).float().mean().item() * 100
                print(f"Batch {step+1}/{len(train_dataloader)}: Loss = {loss.item():.4f}, Acc = {acc:.2f}%", end='\r')
        avg_train_loss = total_loss / len(train_dataloader)
        if val_dataloader is not None:
            val_loss, val_accuracy = evaluate(model, val_dataloader)
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    model.eval()
    val_accuracy = []
    val_loss = []
    for batch in val_dataloader:
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            logits = model(b_input_ids)
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())
        preds = torch.argmax(logits, dim=1).flatten()
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    return val_loss, val_accuracy

# Model evaluation

In [23]:
# CNN-rand: Word vectors are randomly initialized.
set_seed(42)
cnn_rand, optimizer = initialize_model(vocab_size=len(word2idx),
                                      embed_dim=300,
                                      learning_rate=0.25,
                                      dropout=0.5)
train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=3)

# CNN-static: fastText pretrained word vectors are used and freezed during training.
set_seed(42)
cnn_static, optimizer = initialize_model(pretrained_embedding=embeddings,
                                        freeze_embedding=True,
                                        learning_rate=0.25,
                                        dropout=0.5)
train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=3)

# CNN-non-static: fastText pretrained word vectors are fine-tuned during training.
set_seed(42)
cnn_non_static, optimizer = initialize_model(pretrained_embedding=embeddings,
                                            freeze_embedding=False,
                                            learning_rate=0.25,
                                            dropout=0.5)
train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=3)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


Epoch 1:   3%|▎         | 5/156 [00:30<15:13,  6.05s/it]

Epoch 1:   6%|▋         | 10/156 [01:00<14:28,  5.95s/it]

Epoch 1:  10%|▉         | 15/156 [01:29<13:57,  5.94s/it]

Epoch 1:  13%|█▎        | 20/156 [01:58<13:14,  5.84s/it]

...

Epoch 3:  93%|█████████▎| 145/156 [14:48<01:07,  6.14s/it]

Epoch 3:  96%|█████████▌| 150/156 [15:19<00:36,  6.04s/it]

Epoch 3:  99%|█████████▉| 155/156 [15:49<00:06,  6.08s/it]

Epoch 3: 100%|██████████| 156/156 [15:51<00:00,  6.10s/it]


   3    | 2995.716349  | 875.869846 |   99.78   |  969.03  


Training complete! Best accuracy: 100.00%.


In [24]:
import torch

# Зберегти модель CNN-rand
torch.save(cnn_rand.state_dict(), "cnn_rand.pth")

# Зберегти модель CNN-static
torch.save(cnn_static.state_dict(), "cnn_static.pth")

# Зберегти модель CNN-non-static
torch.save(cnn_non_static.state_dict(), "cnn_non_static.pth")

# Test predictions

In [ ]:
cnn_rand.load_state_dict(torch.load("cnn_rand.pth", map_location=device))
cnn_static.load_state_dict(torch.load("cnn_static.pth", map_location=device))
cnn_non_static.load_state_dict(torch.load("cnn_non_static.pth", map_location=device))

cnn_rand.eval()
cnn_static.eval()
cnn_non_static.eval()

def test_model(text, model, max_len=max_len):
    tokens = word_tokenize(text.lower())
    padded_tokens = tokens + ['<pad>'] * (max_len - len(tokens))
    input_id = [word2idx.get(token, word2idx['<unk>']) for token in padded_tokens]
    input_id = torch.tensor(input_id).unsqueeze(dim=0).to(device)
    with torch.no_grad():
        logits = model(input_id)
        probs = torch.softmax(logits, dim=1).squeeze(dim=0)
    print(f"Spam: {probs[1]*100:.2f}%, Not spam: {probs[0]*100:.2f}%")

test_texts = [
    "Congratulations! You have won a free ticket.",
    "Hi, can we meet tomorrow at 10am?",
    "URGENT! Your account has been compromised. Click here to reset your password.",
    "This is not spam, just checking in.",
    "You are selected for a $1000 gift card. Reply YES to claim.",
    "Let's have lunch next week.",
    "Limited time offer! Buy now and save 50%.",
    "Please review the attached document and let me know your thoughts.",
    "Call this number to get your prize.",
    "See you at the meeting later.",
    "Сьогодні пара з Обробки природної мови.",
    "Вітаю! Ви виграли безкоштовний квиток.",
    "Привіт, можемо зустрітися завтра о 10 ранку?",
    "Терміново! Ваш обліковий запис був зламаний. Натисніть тут, щоб скинути пароль."
]

for text in test_texts:
    print(f"\nТекст: {text}")
    print("CNN-rand:")
    test_model(text, cnn_rand)
    print("CNN-static:")
    test_model(text, cnn_static)
    print("CNN-non-static:")
    test_model(text, cnn_non_static)

C:\Users\kleot\AppData\Local\Temp\ipykernel_6452\2735542119.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn_rand.load_state_dict(torch.load("cnn_rand.pth", map_locat


Текст: Congratulations! You have won a free ticket.
CNN-rand:
Spam: 100.00%, Not spam: 0.00%
CNN-static:
Spam: 100.00%, Not spam: 0.00%
CNN-non-static:
Spam: 100.00%, Not spam: 0.00%

Текст: Hi, can we meet tomorrow at 10am?
CNN-rand:
Spam: 0.00%, Not spam: 100.00%
CNN-static:
Spam: 100.00%, Not spam: 0.00%
CNN-non-static:
Spam: 100.00%, Not spam: 0.00%

Текст: URGENT! Your account has been compromised. Click here to reset your password.
CNN-rand:
Spam: 0.00%, Not spam: 100.00%
CNN-static:
Spam: 100.00%, Not spam: 0.00%
CNN-non-static:
Spam: 100.00%, Not spam: 0.00%

Текст: This is not spam, just checking in.
CNN-rand:
Spam: 0.00%, Not spam: 100.00%
CNN-static:
Spam: 100.00%, Not spam: 0.00%
CNN-non-static:
Spam: 100.00%, Not spam: 0.00%

Текст: You are selected for a $1000 gift card. Reply YES to claim.
CNN-rand:
Spam: 0.00%, Not spam: 100.00%
CNN-static:
Spam: 100.00%, Not spam: 0.00%
CNN-non-static:
Spam: 0.00%, Not spam: 100.00%

Текст: Let's have lunch next week.
CNN-rand:
Spam: 